Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All) to avoid typical problems with Jupyter notebooks. **Unfortunately, this does not work with Chrome right now, you will also need to reload the tab in Chrome afterwards**.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Please put your name here:

In [ ]:
NAME = "Aymane Hachcham"

---

# Probabilistic Latent Semantic Indexing

Now we will implement latent semantic indexing (LSI)

In [1]:
### Load the input data - do not modify
import json, gzip, numpy as np
raw = json.load(gzip.open("/data/simpsonswiki.json.gz", "rt", encoding="utf-8"))
titles, texts, classes = [x["title"] for x in raw], [x["text"] for x in raw], [x["c"] for x in raw]

In [2]:
### This cell reduces the data set size for the autograder tests - do not modify

In [3]:
### Vectorize the text - do not modify
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words="english", min_df=5)
counts = cvect.fit_transform(texts)
vocabulary = cvect.get_feature_names_out()

In [4]:
import pandas as pd
pd.DataFrame.sparse.from_spmatrix(counts)

,0,1,2,3,4,5,6,7,8,9,...,14143,14144,14145,14146,14147,14148,14149,14150,14151,14152
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10121,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10123,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Explore your result

Explore the result: write a function to determine the most important words for each factor, and the most relevant documents.

**COPY your code from the first file here** (this is a rare case where copying is okay)

In [5]:
def most_important(vocabulary, factor, k=10):
    """Most important words for each factor"""
    # YOUR CODE HERE
    indices_max_values = np.argpartition(factor, -k)[-k:]
    list_vocabs = [vocabulary[i] for i in indices_max_values]
    return list_vocabs

def most_relevant(assignment, k=5):
    """Most relevant documents for each factor (return document indexes)"""
    # YOUR CODE HERE
    indices_max_values = np.argpartition(assignment, -k)[-k:]
    return indices_max_values

def explain(vocabulary, titles=None, classes=None, factors=None, assignment=None, weights=None):
    """Print an explanation for each factor.
       If weights is None, use the relative share of the assignment weights.
       Print the ARI when assigning each document to its maximum only."""
    from sklearn.metrics import adjusted_rand_score
    for i, f in enumerate(factors):
        print('For the Factor: {}, these are the following results'.format(i))
        important_vocabs = most_important(vocabulary, f)
        print('The most relevant words in this topic are: ')
        print('-------------------------------------------------------')
        print('\n')
        print(important_vocabs)
        important_docs = most_relevant(assignment)
        print('-------------------------------------------------------')
        print('\n')
        print('The most relevant documents belonging to this topic are: ')
        print([titles[i] for fact in important_docs for i in fact])
        print('\n')
        print('Their respective classes are ')
        print([classes[i] for fact in important_docs for i in fact])
        if weights is not None:
            factor_weight = weights[i]
            print('-------------------------------------------------------')
            print('\n')
            print('The Weight factor for this topic is {}'.format(factor_weight))
        print('#################################################################')

## Implement probabilistic Latent Semantic Indexing

Implement pLSI using the non-negative matrix factorization function of sklearn. Make sure to choose appropriate parameters to use KL divergence -- it is not sufficient to use defaults!

In [6]:
# Implement pLSI here using NMF
# Can only use raw term counts, because it is a probabilistic graphical model
def plsi(counts, k):
    """Probabilistic Latent Semantic Indexing. Return the factors and document assignment"""
    from sklearn.decomposition import NMF
    model = NMF(k, random_state=1, beta_loss="kullback-leibler", solver="mu", max_iter=1000, l1_ratio=0.5)

    # Train an NMF model for given Document Term Matrix 'V':
    # Extract the document-topic matrix 'W'
    assignment = model.fit_transform(counts)
    factors = model.components_

    return factors, assignment

In [7]:
### Automatic tests. You do not need to understand or modify this code.
from unittest.mock import patch
with patch('gensim.models.lsimodel.LsiModel') as mock_2:
    _tmp = plsi(counts, 2)
    assert len(_tmp) == 2, "Incomplete result"
    assert _tmp[0].shape == (2, counts.shape[1]), "Factor shape is not correct."
    assert _tmp[1].shape == (counts.shape[0], 2), "Assignment shape is not correct."
    assert not mock_2.called, "You were supposed to use sklearn here, not gensim."
del _tmp

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


In [8]:
### This cell contains additional tests. You do not need to modify this cell.

In [9]:
# Explore your result. These must be meaningful topics!
plsi_factors, plsi_assignment = plsi(counts, 6)
explain(vocabulary, titles, classes, plsi_factors, plsi_assignment)

For the Factor: 0, these are the following results
The most relevant words in this topic are: 
-------------------------------------------------------


['house', 'family', 'moe', 'tells', 'says', 'episode', 'lisa', 'marge', 'bart', 'homer']
-------------------------------------------------------


The most relevant documents belonging to this topic are: 
['Bart Jumps', 'Burp Contest', 'Good Night', 'Babysitting Maggie', 'The Pacifier', 'Watching Television', 'Watching Television', 'Burp Contest', 'Bart Jumps', 'Babysitting Maggie', 'The Pacifier', 'Good Night', 'Good Night', 'Burp Contest', 'Bart Jumps', 'Babysitting Maggie', 'The Pacifier', 'Watching Television', 'Burp Contest', 'Bart Jumps', 'Watching Television', 'Babysitting Maggie', 'The Pacifier', 'Good Night', 'Watching Television', 'Burp Contest', 'Bart Jumps', 'Babysitting Maggie', 'The Pacifier', 'Good Night']


Their respective classes are 
['Episodes', 'Episodes', 'Episodes', 'Episodes', 'Episodes', 'Episodes', 'Episodes',

In [ ]:
### This cell contains additional tests. You do not need to modify this cell.